In [9]:
import os
os.chdir('/content/drive/Shareddrives/NLP 대회/new')

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!pip install textstat
!pip install fasttext
!pip install xgboost
!pip install nltk

In [12]:
import pandas as pd
import numpy as np

In [13]:
train = pd.read_csv('train.csv', encoding='utf-8')
test = pd.read_csv('test_x.csv', encoding='utf-8')

In [14]:
#사람 이름 구하기
#필요 import
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk import ne_chunk, tree2conlltags
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('maxent_ne_chunker')
nltk.download('words')

#코드
def get_persons(text):
    a=ne_chunk(pos_tag(word_tokenize(text)))  
    b=tree2conlltags(a)  
    c=[list(x) for x in b]
    #token화 하고 품사 태깅 후에 ne_chunk, tree2conlltags, 리스트 안에 리스트('단어','품사','사람여부*')가 들어가도록
    #*원래는 사람여부는 아닌데 아래에서 PERSON 값을 가질 경우 따로 처리함
    
    
    person_vector=[]
    person_name=[]
    for i, word in enumerate(c):
        if("PERSON" in word[2]):
            person_name.append(word[0])
        else:
            if(len(person_name)!=0):
                person_vector.append("".join(person_name))
            person_name=[]
        
        if(i==len(c)-1 and len(person_name)!=0):
            person_vector.append("".join(person_name))
    return person_vector
    #list에 번호를 매겨서 그 index를 i로 하고, 그 값들을 word라고 할 때, 만약 리스트('단어','품사','사람여부*')의 사람여부 항목이
    #사람으로 표시되어 있으면, person_name에 그 단어를 추가한다.
    #사람으로 표시되어 있지 않으면, person_name을 확인하여 비어있지 않으면 그 단어를 person_vector에 추가하고 person_name을 초기화
    #만약 i(index)가 list의 길이-1 즉, 마지막 인덱스이고, person_name이 비어있지 않으면, 마지막 단어를 person_vector에 추가한다.
    #최종 person_vector를 출력

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [15]:
#저자의 글 모으기
# author번호가 *인 text를 쭉 늘어놓은 리스트
text_author_0 = " ".join(list(train['text'][train['author']==0]))
text_author_1 = " ".join(list(train['text'][train['author']==1]))
text_author_2 = " ".join(list(train['text'][train['author']==2]))
text_author_3 = " ".join(list(train['text'][train['author']==3]))
text_author_4 = " ".join(list(train['text'][train['author']==4]))

In [16]:
#저자의 글에 나오는 등장인물 이름
# *번 저자의 글에 나오는 사람이름의 집합(순서 없음)
persons_author_0 = set(get_persons(text_author_0))
print("0 completed")
persons_author_1 = set(get_persons(text_author_1))
print("1 completed")
persons_author_2 = set(get_persons(text_author_2))
print("2 completed")
persons_author_3 = set(get_persons(text_author_3))
print("3 completed")
persons_author_4 = set(get_persons(text_author_4))
print("4 completed")

0 completed
1 completed
2 completed
3 completed
4 completed


In [17]:
#비트연산자를 이용한 함수
# a&b 는 a의 이진수 값과 b의 이진수 값을 비교하여 
# 자리수가 모두 1이면 1, 아니면 0을 output으로 하는 이진수의 십진수 값을 반환하는 연산자 (and와 유사)

# a|b 는 a의 이진수 값과 b의 이진수 값을 비교하여 
# 자리수가 하나라도 1이면 1, 아니면 0을 output으로 하는 이진수의 십진수 값을 반환하는 연산자 (or과 유사)

# 아래 함수는 두 input의 교집합의 크기를 합집합의 크기로 나눠주는 함수. 항상 <=1


def jaccard(a,b):
    return len(a&b)/len(a|b)


In [18]:
from tqdm import tqdm
tqdm.pandas()

# train과 test 데이터셋에서 각 행의 'text'에 나오는 사람이름이 *번 저자의 글에 나오는 사람이름과 얼마나 유사한지 측정하는 변수
# 두 세트의 겹치는 이름 개수 / 두 텍스트 전체 이름 개수

train["persons_0"]=train["text"].progress_apply(lambda x:jaccard(set(get_persons(x)),persons_author_0)) 


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 54879/54879 [15:02<00:00, 60.77it/s]


In [19]:
train["persons_1"]=train["text"].progress_apply(lambda x:jaccard(set(get_persons(x)),persons_author_1)) 
train["persons_2"]=train["text"].progress_apply(lambda x:jaccard(set(get_persons(x)),persons_author_2)) 
train["persons_3"]=train["text"].progress_apply(lambda x:jaccard(set(get_persons(x)),persons_author_3)) 
train["persons_4"]=train["text"].progress_apply(lambda x:jaccard(set(get_persons(x)),persons_author_4)) 

test["persons_0"]=test["text"].progress_apply(lambda x:jaccard(set(get_persons(x)),persons_author_0)) 
test["persons_1"]=test["text"].progress_apply(lambda x:jaccard(set(get_persons(x)),persons_author_1)) 
test["persons_2"]=test["text"].progress_apply(lambda x:jaccard(set(get_persons(x)),persons_author_2)) 
test["persons_3"]=test["text"].progress_apply(lambda x:jaccard(set(get_persons(x)),persons_author_3)) 
test["persons_4"]=test["text"].progress_apply(lambda x:jaccard(set(get_persons(x)),persons_author_4)) 

100%|██████████| 19617/19617 [11:02<00:00, 29.63it/s]


In [20]:
import string

In [21]:
#stopwords
from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words("english"))

In [22]:
train["num_words"] = train["text"].progress_apply(lambda x: len(str(x).split()))
train["num_unique_words"] = train["text"].progress_apply(lambda x: len(set(str(x).split())))
train["num_chars"] = train["text"].progress_apply(lambda x: len(str(x)))
train["num_stopwords"] = train["text"].progress_apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
#train["num_punctuations"] =train['text'].progress_apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
train["num_words_upper"] = train["text"].progress_apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
train["mean_word_len"] = train["text"].progress_apply(lambda x: np.mean([len(w) for w in str(x).split()]))
train["num_words_title"] = train["text"].progress_apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

100%|██████████| 54879/54879 [00:00<00:00, 139484.04it/s]


In [23]:
test["num_words"] = test["text"].progress_apply(lambda x: len(str(x).split()))
test["num_unique_words"] = test["text"].progress_apply(lambda x: len(set(str(x).split())))
test["num_chars"] = test["text"].progress_apply(lambda x: len(str(x)))
test["num_stopwords"] = test["text"].progress_apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
#test["num_punctuations"] =test['text'].progress_apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test["num_words_upper"] = test["text"].progress_apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test["mean_word_len"] = test["text"].progress_apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test["num_words_title"] = test["text"].progress_apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

100%|██████████| 19617/19617 [00:00<00:00, 72244.79it/s]


In [24]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def sentiment_nltk(text):
    res = SentimentIntensityAnalyzer().polarity_scores(text)
    return res['compound']

train["sentiment"]=train["text"].progress_apply(sentiment_nltk)
test["sentiment"]=test["text"].progress_apply(sentiment_nltk)

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
100%|██████████| 19617/19617 [02:38<00:00, 123.56it/s]


In [25]:
#자주 사용하는 품사 정보

def fraction_noun(text):
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    word_count = text_splited.__len__()
    if word_count==0:
        return 0
    else:
        pos_list = nltk.pos_tag(text_splited)
        noun_count = len([w for w in pos_list if w[1] in ('NN','NNP','NNPS','NNS')])
    
        return (noun_count/word_count)

def fraction_adj(text):
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    word_count = text_splited.__len__()
    if word_count==0:
        return 0
    else:
        pos_list = nltk.pos_tag(text_splited)
        adj_count = len([w for w in pos_list if w[1] in ('JJ','JJR','JJS')])
    
        return (adj_count/word_count)  

def fraction_verbs(text):
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    word_count = text_splited.__len__()
    if word_count==0:
        return 0
    else:
        pos_list = nltk.pos_tag(text_splited)
        verbs_count = len([w for w in pos_list if w[1] in ('VB','VBD','VBG','VBN','VBP','VBZ')])
    
        return (verbs_count/word_count)  


In [26]:
train['noun'] = train["text"].progress_apply(lambda x: fraction_noun(x))
train['adj'] = train["text"].progress_apply(lambda x: fraction_adj(x))
train['verbs'] = train["text"].progress_apply(lambda x: fraction_verbs(x))

100%|██████████| 54879/54879 [01:43<00:00, 528.59it/s]


In [27]:
test['noun'] = test["text"].progress_apply(lambda x: fraction_noun(x))
test['adj'] = test["text"].progress_apply(lambda x: fraction_adj(x))
test['verbs'] = test["text"].progress_apply(lambda x: fraction_verbs(x))

100%|██████████| 19617/19617 [01:13<00:00, 265.62it/s]


In [28]:
import re

#clean
def clean(train,test):
    train['words'] = [re.sub("[^a-zA-Z]"," ", data).lower().split() for data in train['text']]
    test['words'] = [re.sub("[^a-zA-Z]"," ", data).lower().split() for data in test['text']]
    return train,test
train, test = clean(train, test)


In [29]:
train.head(5)

,index,text,author,persons_0,persons_1,persons_2,persons_3,persons_4,num_words,num_unique_words,num_chars,num_stopwords,num_words_upper,mean_word_len,num_words_title,sentiment,noun,adj,verbs,words
0,0,"He was almost choking. There was so much, so m...",3,0.000000,0.00000,0.000000,0.000000,0.000000,46,39,240,25,0,4.239130,4,0.3064,0.173913,0.065217,0.217391,"[he, was, almost, choking, there, was, so, muc..."
1,1,"“Your sister asked for it, I suppose?”",2,0.000000,0.00000,0.000000,0.000000,0.000000,7,7,38,2,1,4.571429,2,0.0000,0.142857,0.142857,0.285714,"[your, sister, asked, for, it, i, suppose]"
2,2,"She was engaged one day as she walked, in per...",1,0.002014,0.00312,0.000644,0.001033,0.001182,57,50,320,26,0,4.614035,4,0.7351,0.192982,0.017544,0.280702,"[she, was, engaged, one, day, as, she, walked,..."
3,3,"The captain was in the porch, keeping himself ...",4,0.000000,0.00000,0.000644,0.000000,0.001183,58,49,319,26,0,4.517241,7,0.6908,0.293103,0.051724,0.189655,"[the, captain, was, in, the, porch, keeping, h..."
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3,0.001007,0.00156,0.000644,0.001034,0.001183,39,36,228,16,0,4.871795,4,0.1984,0.333333,0.025641,0.205128,"[have, mercy, gentlemen, odin, flung, up, his,..."


In [30]:
test.head(5)

,index,text,persons_0,persons_1,persons_2,persons_3,persons_4,num_words,num_unique_words,num_chars,num_stopwords,num_words_upper,mean_word_len,num_words_title,sentiment,noun,adj,verbs,words
0,0,“Not at all. I think she is one of the most ch...,0.000000,0.00000,0.000644,0.000000,0.000000,89,68,456,49,5,4.134831,9,0.9440,0.179775,0.123596,0.191011,"[not, at, all, i, think, she, is, one, of, the..."
1,1,"""No,"" replied he, with sudden consciousness, ""...",0.000000,0.00000,0.000000,0.000000,0.000000,43,36,221,21,5,4.162791,5,0.5739,0.139535,0.069767,0.232558,"[no, replied, he, with, sudden, consciousness,..."
2,2,As the lady had stated her intention of scream...,0.001007,0.00156,0.000644,0.001034,0.001183,64,55,375,32,0,4.875000,2,-0.7050,0.218750,0.078125,0.187500,"[as, the, lady, had, stated, her, intention, o..."
3,3,“And then suddenly in the silence I heard a so...,0.000000,0.00000,0.000000,0.000000,0.000000,240,150,1218,121,11,4.079167,21,-0.9787,0.225000,0.041667,0.158333,"[and, then, suddenly, in, the, silence, i, hea..."
4,4,His conviction remained unchanged. So far as I...,0.000000,0.00000,0.000000,0.000000,0.000000,91,71,510,44,4,4.615385,8,0.9414,0.164835,0.098901,0.186813,"[his, conviction, remained, unchanged, so, far..."


In [31]:
#import
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import naive_bayes, metrics, model_selection

#모델들 

def runLR(train_X,train_y,test_X,test_y,test_X2):
    model=LogisticRegression(max_iter=1000)
    model.fit(train_X,train_y)
    pred_test_y=model.predict_proba(test_X)
    pred_test_y2=model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def runSGD(train_X,train_y,test_X,test_y,test_X2):
    model=SGDClassifier(loss='log')
    model.fit(train_X,train_y)
    pred_test_y=model.predict_proba(test_X)
    pred_test_y2=model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def runRF(train_X,train_y,test_X,test_y,test_X2):
    model=RandomForestClassifier()
    model.fit(train_X,train_y)
    pred_test_y=model.predict_proba(test_X)
    pred_test_y2=model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def runMLP(train_X,train_y,test_X,test_y,test_X2):
    model=MLPClassifier()
    model.fit(train_X,train_y)
    pred_test_y=model.predict_proba(test_X)
    pred_test_y2=model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def runDT(train_X,train_y,test_X,test_y,test_X2):
    model=DecisionTreeClassifier()
    model.fit(train_X,train_y)
    pred_test_y=model.predict_proba(test_X)
    pred_test_y2=model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def runMNB(train_X,train_y,test_X,test_y,test_X2):
    model=naive_bayes.MultinomialNB()
    model.fit(train_X,train_y)
    pred_test_y=model.predict_proba(test_X)
    pred_test_y2=model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

# Tfidf + LR
tfidf_vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=50)
train_tfidf = tfidf_vec.fit_transform(train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test['text'].values.tolist())
train_y = train['author']

cv_scores=[]
cols_to_drop=['text','index']
train_X = train.drop(cols_to_drop+['author'], axis=1)
train_y=train['author']
test_X = test.drop(cols_to_drop, axis=1)
pred_train=np.zeros([train.shape[0],5])
pred_full_test = 0

cv = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)

for dev_index, val_index in tqdm(cv.split(train_X,train_y)):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runLR(dev_X, dev_y, val_X, val_y,test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

train["tfidf_LR_0"] = pred_train[:,0]
train["tfidf_LR_1"] = pred_train[:,1]
train["tfidf_LR_2"] = pred_train[:,2]
train["tfidf_LR_3"] = pred_train[:,3]
train["tfidf_LR_4"] = pred_train[:,4]
test["tfidf_LR_0"] = pred_full_test[:,0]
test["tfidf_LR_1"] = pred_full_test[:,1]
test["tfidf_LR_2"] = pred_full_test[:,2]
test["tfidf_LR_3"] = pred_full_test[:,3]
test["tfidf_LR_4"] = pred_full_test[:,4]

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
5it [01:55, 23.19s/it]

Mean cv score :  0.685600629942216


In [33]:
#Tfidf + SGD
cv_scores=[]
cols_to_drop=['text','index']
train_X = train.drop(cols_to_drop+['author'], axis=1)
train_y=train['author']
test_X = test.drop(cols_to_drop, axis=1)
pred_train=np.zeros([train.shape[0],5])
pred_full_test = 0

cv = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)

for dev_index, val_index in cv.split(train_X,train_y):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runSGD(dev_X, dev_y, val_X, val_y,test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

train["tfidf_SGD_0"] = pred_train[:,0]
train["tfidf_SGD_1"] = pred_train[:,1]
train["tfidf_SGD_2"] = pred_train[:,2]
train["tfidf_SGD_3"] = pred_train[:,3]
train["tfidf_SGD_4"] = pred_train[:,4]
test["tfidf_SGD_0"] = pred_full_test[:,0]
test["tfidf_SGD_1"] = pred_full_test[:,1]
test["tfidf_SGD_2"] = pred_full_test[:,2]
test["tfidf_SGD_3"] = pred_full_test[:,3]
test["tfidf_SGD_4"] = pred_full_test[:,4]

Mean cv score :  0.8855549507495477


In [34]:
#Tfidf + RF
cv_scores=[]
cols_to_drop=['text','index']
train_X = train.drop(cols_to_drop+['author'], axis=1)
train_y=train['author']
test_X = test.drop(cols_to_drop, axis=1)
pred_train=np.zeros([train.shape[0],5])
pred_full_test = 0

cv = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)

for dev_index, val_index in cv.split(train_X,train_y):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runRF(dev_X, dev_y, val_X, val_y,test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

train["tfidf_RF_0"] = pred_train[:,0]
train["tfidf_RF_1"] = pred_train[:,1]
train["tfidf_RF_2"] = pred_train[:,2]
train["tfidf_RF_3"] = pred_train[:,3]
train["tfidf_RF_4"] = pred_train[:,4]
test["tfidf_RF_0"] = pred_full_test[:,0]
test["tfidf_RF_1"] = pred_full_test[:,1]
test["tfidf_RF_2"] = pred_full_test[:,2]
test["tfidf_RF_3"] = pred_full_test[:,3]
test["tfidf_RF_4"] = pred_full_test[:,4]

Mean cv score :  0.9601250937183746


In [ ]:
#Tfidf+MLP
cv_scores=[]
cols_to_drop=['text','index']
train_X = train.drop(cols_to_drop+['author'], axis=1)
train_y=train['author']
test_X = test.drop(cols_to_drop, axis=1)
pred_train=np.zeros([train.shape[0],5])
pred_full_test = 0

cv = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)

for dev_index, val_index in tqdm(cv.split(train_X,train_y)):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMLP(dev_X, dev_y, val_X, val_y,test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

train["tfidf_MLP_0"] = pred_train[:,0]
train["tfidf_MLP_1"] = pred_train[:,1]
train["tfidf_MLP_2"] = pred_train[:,2]
train["tfidf_MLP_3"] = pred_train[:,3]
train["tfidf_MLP_4"] = pred_train[:,4]
test["tfidf_MLP_0"] = pred_full_test[:,0]
test["tfidf_MLP_1"] = pred_full_test[:,1]
test["tfidf_MLP_2"] = pred_full_test[:,2]
test["tfidf_MLP_3"] = pred_full_test[:,3]
test["tfidf_MLP_4"] = pred_full_test[:,4]

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
1it [07:16, 436.40s/it]

In [ ]:
#Tfidf+Decision Tree
cv_scores=[]
cols_to_drop=['text','index']
train_X = train.drop(cols_to_drop+['author'], axis=1)
train_y=train['author']
test_X = test.drop(cols_to_drop, axis=1)
pred_train=np.zeros([train.shape[0],5])
pred_full_test = 0

cv = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)


for dev_index, val_index in tqdm(cv.split(train_X,train_y)):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runDT(dev_X, dev_y, val_X, val_y,test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

train["tfidf_DT_0"] = pred_train[:,0]
train["tfidf_DT_1"] = pred_train[:,1]
train["tfidf_DT_2"] = pred_train[:,2]
train["tfidf_DT_3"] = pred_train[:,3]
train["tfidf_DT_4"] = pred_train[:,4]
test["tfidf_DT_0"] = pred_full_test[:,0]
test["tfidf_DT_1"] = pred_full_test[:,1]
test["tfidf_DT_2"] = pred_full_test[:,2]
test["tfidf_DT_3"] = pred_full_test[:,3]
test["tfidf_DT_4"] = pred_full_test[:,4]


In [ ]:
# ngram=2
cvec_vec=CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=50)
cvec_vec.fit(train['text'].values.tolist())
train_cvec = cvec_vec.transform(train['text'].values.tolist())
test_cvec = cvec_vec.transform(test['text'].values.tolist())

# CountV + LR
cv_scores=[]
cols_to_drop=['text','index']
train_X = train.drop(cols_to_drop+['author'], axis=1)
train_y=train['author']
test_X = test.drop(cols_to_drop, axis=1)
pred_train=np.zeros([train.shape[0],5])
pred_full_test = 0

cv = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)

for dev_index, val_index in tqdm(cv.split(train_X,train_y)):
    dev_X, val_X = train_cvec[dev_index], train_cvec[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runLR(dev_X, dev_y, val_X, val_y,test_cvec)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

train["cvec_LR_0"] = pred_train[:,0]
train["cvec_LR_1"] = pred_train[:,1]
train["cvec_LR_2"] = pred_train[:,2]
train["cvec_LR_3"] = pred_train[:,3]
train["cvec_LR_4"] = pred_train[:,4]
test["cvec_LR_0"] = pred_full_test[:,0]
test["cvec_LR_1"] = pred_full_test[:,1]
test["cvec_LR_2"] = pred_full_test[:,2]
test["cvec_LR_3"] = pred_full_test[:,3]
test["cvec_LR_4"] = pred_full_test[:,4]

In [ ]:
!pip install textstat

In [ ]:
from textstat import flesch_reading_ease

train['mean_word_len']=train['text'].apply(lambda x:np.mean([len(w) for w in str(x).split()]))
test['mean_word_len']=test['text'].apply(lambda x:np.mean([len(w) for w in str(x).split()]))
train["chars_between_comma"] = train["text"].apply(lambda x: np.mean([len(chunk) for chunk in str(x).split(",")]))/train["num_chars"]
test["chars_between_comma"] = test["text"].apply(lambda x: np.mean([len(chunk) for chunk in str(x).split(",")]))/test["num_chars"]
train['ease']=train['text'].apply(flesch_reading_ease)
test['ease']=test['text'].apply(flesch_reading_ease)


In [ ]:
stop_trn = [x for x in train['words']]
train['stop_word'] = [len([word for word in sentence if word in stopwords.words('english')])*100.0/(len(sentence)+1) for sentence in stop_trn]
print('process completed ')

stop_tst = [x for x in test['words']]
test['stop_word'] = [len([word for word in sentence if word in stopwords.words('english')])*100.0/(len(sentence)+1) for sentence in stop_tst] 
print('process completed ')

In [ ]:
def extract_features(df):
    df['len'] = df['text'].apply(lambda x: len(x))
    df['n_words'] = df['text'].apply(lambda x: len(x.split(' ')))
    df['n_.'] = df['text'].str.count('\.')
    df['n_...'] = df['text'].str.count('\...')
    df['n_,'] = df['text'].str.count('\,')
    df['n_:'] = df['text'].str.count('\:')
    df['n_;'] = df['text'].str.count('\;')
    df['n_-'] = df['text'].str.count('\-')
    df['n_?'] = df['text'].str.count('\?')
    df['n_!'] = df['text'].str.count('\!')
    df['n_\''] = df['text'].str.count('\'')
    df['n_"'] = df['text'].str.count('\"')

    # First words in a sentence
    df['n_The '] = df['text'].str.count('The ')
    df['n_I '] = df['text'].str.count('I ')
    df['n_It '] = df['text'].str.count('It ')
    df['n_He '] = df['text'].str.count('He ')
    df['n_Me '] = df['text'].str.count('Me ')
    df['n_She '] = df['text'].str.count('She ')
    df['n_We '] = df['text'].str.count('We ')
    df['n_They '] = df['text'].str.count('They ')
    df['n_You '] = df['text'].str.count('You ')
    df['n_the'] = df['words'].str.count('the ')
    df['n_ a '] = df['words'].str.count(' a ')
    df['n_appear'] = df['words'].str.count('appear')
    df['n_little'] = df['words'].str.count('little')
    df['n_was '] = df['words'].str.count('was ')
    df['n_one '] = df['words'].str.count('one ')
    df['n_two '] = df['words'].str.count('two ')
    df['n_three '] = df['words'].str.count('three ')
    df['n_ten '] = df['words'].str.count('ten ')
    df['n_is '] = df['words'].str.count('is ')
    df['n_are '] = df['words'].str.count('are ')
    df['n_ed'] = df['words'].str.count('ed ')
    df['n_however'] = df['words'].str.count('however')
    df['n_ to '] = df['words'].str.count(' to ')
    df['n_into'] = df['words'].str.count('into')
    df['n_about '] = df['words'].str.count('about ')
    df['n_th'] = df['words'].str.count('th')
    df['n_er'] = df['words'].str.count('er')
    df['n_ex'] = df['words'].str.count('ex')
    df['n_an '] = df['words'].str.count('an ')
    df['n_ground'] = df['words'].str.count('ground')
    df['n_any'] = df['words'].str.count('any')
    df['n_silence'] = df['words'].str.count('silence')
    df['n_wall'] = df['words'].str.count('wall')

    df.drop(['words'], axis=1, inplace=True)
    
print('Processing train...')
extract_features(train)
print('Processing test...')
extract_features(test)

In [ ]:
from nltk.corpus import stopwords
tfidf_vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=50)
svd_tfidf = tfidf_vec.fit_transform(train['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test['text'].values.tolist())
train_y = train['author']

In [ ]:
from nltk.corpus import stopwords
tfidf_vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=50)
svd_tfidf = tfidf_vec.fit_transform(train['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test['text'].values.tolist())
train_y = train['author']

In [ ]:
#Tfidf + MNB
cv_scores=[]
cols_to_drop=['text','index']
train_X = train.drop(cols_to_drop+['author'], axis=1)
train_y=train['author']
test_X = test.drop(cols_to_drop, axis=1)
pred_train=np.zeros([train.shape[0],5])
pred_full_test = 0

cv = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)

for dev_index, val_index in cv.split(train_X,train_y):
    dev_X, val_X = svd_tfidf[dev_index], svd_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y,test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

X_train["tfidf_words_nb_cvec_0"] = pred_train[:,0]
X_train["tfidf_words_nb_cvec_1"] = pred_train[:,1]
X_train["tfidf_words_nb_cvec_2"] = pred_train[:,2]
X_train["tfidf_words_nb_cvec_3"] = pred_train[:,3]
X_train["tfidf_words_nb_cvec_4"] = pred_train[:,4]
X_test["tfidf_words_nb_cvec_0"] =  pred_full_test[:,0]
X_test["tfidf_words_nb_cvec_1"] =  pred_full_test[:,1]
X_test["tfidf_words_nb_cvec_2"] =  pred_full_test[:,2]
X_test["tfidf_words_nb_cvec_3"] =  pred_full_test[:,3]
X_test["tfidf_words_nb_cvec_4"] =  pred_full_test[:,4]

In [ ]:
n_comp = 5
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(train_tfidf)

train_svd = svd_obj.transform(train_tfidf)
test_svd = svd_obj.transform(test_tfidf)

from sklearn import preprocessing
scl = preprocessing.StandardScaler()
scl.fit(train_svd)
train_svd_scl = pd.DataFrame(scl.transform(train_svd))
test_svd_scl = pd.DataFrame(scl.transform(test_svd))

train_svd_scl.columns = ['svd_char_'+str(i) for i in range(n_comp)]
test_svd_scl.columns = ['svd_char_'+str(i) for i in range(n_comp)]
train = pd.concat([train, train_svd_scl], axis=1)
test = pd.concat([test, test_svd_scl], axis=1)

In [ ]:
from sklearn import model_selection
import xgboost as xgb

cols_to_drop = ['index', 'text']
train_X = train.drop(cols_to_drop+['author'], axis=1)
train_y=train['author']
test_index = test['index'].values
test_X = test.drop(cols_to_drop, axis=1)
xgb_preds=[]
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2020)

for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_X.loc[dev_index], train_X.loc[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    dtrain = xgb.DMatrix(dev_X,label=dev_y)
    dvalid = xgb.DMatrix(val_X, label=val_y)
    watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 3
    param['silent'] = 1
    param['num_class'] = 5
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.3
    param['seed'] = 0
    param['tree_method'] = 'gpu_hist'

    model = xgb.train(param, dtrain, 2000, watchlist, early_stopping_rounds=50, verbose_eval=20)

    xgtest2 = xgb.DMatrix(test_X)
    xgb_pred = model.predict(xgtest2, ntree_limit = model.best_ntree_limit)
    xgb_preds.append(list(xgb_pred))


In [ ]:
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(5):
        sum+=xgb_preds[j][i]    
    if(i==0):
        preds=sum/5
    else:
        preds=np.vstack([preds,sum/5])

preds=pd.DataFrame(preds)

preds.to_csv('new_submission.csv')